# 환경 셋업

In [1]:
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

Installing dependencies...


In [4]:
import glob

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
# 코랩환경을 위한 환경 설정, 새로 설치한 fluidsynth 라이브러리를 사용하기 위함.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries...')
from google.colab import files
from magenta.models.music_vae import configs
from magenta.models.music_vae import data,configs, music_vae_train
from magenta.models.music_vae.trained_model import TrainedModel
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae.base_model import MusicVAE
from magenta.models.music_vae.configs import update_config

import numpy as np
import os
import tensorflow.compat.v1 as tf
import note_seq

import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)


Importing libraries...


# 모델 학습하기

In [5]:
def hierdec_4bar_model():
  return MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                  lstm_models.HierarchicalLstmDecoder(
                    lstm_models.GrooveLstmDecoder(),
                    level_lengths=[4, 16],
                    disable_autoregression=True)
                  )

groovae_4bar_config = configs.CONFIG_MAP['groovae_4bar']
groovae_4bar_config = update_config(groovae_4bar_config, 
                                    {'model': hierdec_4bar_model()})


In [6]:
def dataset_fn():
  return data.get_dataset(config=groovae_4bar_config,
                          tf_file_reader=tf.data.TFRecordDataset,
                          is_training=True)

music_vae_train.train(train_dir='model/groovae_4bar/', 
                      config=groovae_4bar_config,
                      dataset_fn=dataset_fn,
                      checkpoints_to_keep=100,
                      keep_checkpoint_every_n_hours=1,
                      num_steps=2000,
                      master='',
                      num_sync_workers=0,
                      num_ps_tasks=0,
                      task=0)

/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:750: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:753: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  initializer=tf.constant_initializer(0.0))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras

In [9]:
def play(note_sequence):
  note_seq.play_sequence(note_sequence, synth=note_seq.fluidsynth)
  pass

groovae_4bar_config = update_config(groovae_4bar_config, 
                                    {'model': hierdec_4bar_model()})
drum_model = TrainedModel(groovae_4bar_config, batch_size=1, 
                          checkpoint_dir_or_path='model/groovae_4bar/model.ckpt-0')
temperature = 0.5 #@param

print('학습되지 않은 모델의 출력')
n = 0
while n < 4:
  try:
    drums_samples = drum_model.sample(n=1, length=64, temperature=temperature)
    play(drums_samples[0])
    n += 1
  except:
    pass
drum_model = TrainedModel(groovae_4bar_config, batch_size=1, 
                          checkpoint_dir_or_path='model/groovae_4bar/model.ckpt-2000')

print('학습된 모델의 출력')
n = 0
while n < 4:
  try:
    drums_samples = drum_model.sample(n=1, length=64, temperature=temperature)
    play(drums_samples[0])
    n += 1
  except:
    pass

학습되지 않은 모델의 출력


학습된 모델의 출력
